In [1]:
import pyspark
from pyspark.context import SparkContext
from pyspark.sql.session import SparkSession
from pyspark.sql.functions import to_timestamp, col, shuffle, rand
from pyspark.sql.types import LongType, DoubleType, StringType, TimestampType, StructType, StructField, DateType, FloatType, IntegerType, ArrayType
import datetime

In [2]:
sc = SparkContext("local")
spark = SparkSession(sc)

21/09/07 21:16:54 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
21/09/07 21:16:55 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [3]:
test_df_schema = StructType([
  StructField('SM_USERNAME', StringType()),
  StructField('SM_RESOURCE', StringType()),
  StructField('SM_TIMESTAMP_TEMP', StringType()), 
  StructField('SM_EVENTID', LongType()), 
  StructField('SM_CLIENTIP', StringType())
])

ans_schema = StructType([
    StructField('window_temp', StructType([  
    StructField('start', StringType()),
    StructField('end', StringType())
  ])),
  StructField('StartTime',StringType()), 
  StructField('EndTime', StringType()),
  StructField('VolOfAllLoginAttempts', LongType()),
  StructField('VolOfAllFailedLogins', LongType()),
  StructField('NumOfReqsToChangePasswords', LongType()),
  StructField('MaxOfFailedLoginsWithSameIPs', LongType()),
  StructField('NumOfIPsLoginMultiAccounts', LongType()),
  StructField('NumOfUsersWithEqualIntervalBtnReqs', LongType()), 
])


In [4]:
def process_test_data(test_data):
    test_df = spark.createDataFrame(test_data, schema=test_df_schema)
    test_df = test_df.withColumn('SM_TIMESTAMP', col('SM_TIMESTAMP_TEMP').cast('timestamp'))
    test_df = test_df.drop('SM_TIMESTAMP_TEMP')
    return test_df

def process_expected_data(expected_data):
    ans_df = spark.createDataFrame(expected_data, schema=ans_schema)
    ans_df = ans_df.withColumn('window', col('window_temp').cast(StructType([StructField('start', TimestampType()),StructField('end', TimestampType())])))
    ans_df = ans_df.drop('window_temp')
    ans_df = ans_df.withColumn('StartTime', col('StartTime').cast(TimestampType()))
    ans_df = ans_df.withColumn('EndTime', col('EndTime').cast(TimestampType()))
    ans_df = ans_df.select('window', 'StartTime', 'EndTime', 'VolOfAllLoginAttempts', 'VolOfAllFailedLogins', 'NumOfReqsToChangePasswords', 'MaxOfFailedLoginsWithSameIPs', \
                       'NumOfIPsLoginMultiAccounts', 'NumOfUsersWithEqualIntervalBtnReqs')
    return ans_df

In [5]:
user2_one_window = [
  ('cn=u1,ou=Identity', '/path/changePassword', '2018-02-01T00:00:00.000+0000', 1, 'IP1'), 
  ('cn=u1,ou=Identity', '/path/changePassword', '2018-02-01T00:00:01.000+0000', 1, 'IP1'), 
  ('cn=u1,ou=Identity', '/path/changePassword', '2018-02-01T00:00:03.000+0000', 1, 'IP1'),
  ('cn=u2,ou=Identity', '/path/changePassword', '2018-02-01T00:00:03.000+0000', 1, 'IP1'),
  ('cn=u2,ou=Identity', '/path/changePassword', '2018-02-01T00:00:06.000+0000', 1, 'IP1') 
]

expected_2user_one_window = [
  
  ({"start": "2018-02-01T00:00:00.000+0000", "end": "2018-02-01T00:15:00.000+0000"}, '2018-02-01T00:00:00.000+0000', '2018-02-01T00:15:00.000+0000', 5, 0, 5, 0, 1, 2)
]

user2_one_window_df = process_test_data(user2_one_window)
expected_2user_one_window_df = process_expected_data(expected_2user_one_window)

In [7]:
user2_one_window_df.write.parquet('/home/vmadmin/PycharmProjects/sw-test/caa-streamworx/data/parquet_data/server_feature_generator_tests/user2_one_window_df.parquet')
expected_2user_one_window_df.write.parquet('/home/vmadmin/PycharmProjects/sw-test/caa-streamworx/data/parquet_data/server_feature_generator_tests/expected_2user_one_window_df.parquet')

In [6]:
user5_one_window = [
  ('cn=u1,ou=Identity', '/path/changePassword', '2018-02-01T00:00:00.000+0000', 6, 'IP1'), 
  ('cn=u2,ou=Identity', '/path/changePassword', '2018-02-01T00:00:01.000+0000', 6, 'IP1'), 
  ('cn=u3,ou=Identity', '/path/changePassword', '2018-02-01T00:00:03.000+0000', 6, 'IP1'),
  ('cn=u4,ou=Identity', '/path/changePassword', '2018-02-01T00:00:03.000+0000', 6, 'IP1'),
  ('cn=u5,ou=Identity', '/path/changePassword', '2018-02-01T00:00:06.000+0000', 6, 'IP1') 
]

expected_5user_one_window = [
  
  ({"start": "2018-02-01T00:00:00.000+0000", "end": "2018-02-01T00:15:00.000+0000"}, '2018-02-01T00:00:00.000+0000', '2018-02-01T00:15:00.000+0000', 5, 5, 5, 5, 1, 0)
]

user5_one_window_df = process_test_data(user5_one_window)
expected_5user_one_window_df = process_expected_data(expected_5user_one_window)

In [8]:
user5_one_window_df.write.parquet('/home/vmadmin/PycharmProjects/sw-test/caa-streamworx/data/parquet_data/server_feature_generator_tests/user5_one_window_df.parquet')
expected_5user_one_window_df.write.parquet('/home/vmadmin/PycharmProjects/sw-test/caa-streamworx/data/parquet_data/server_feature_generator_tests/expected_5user_one_window_df.parquet')

In [9]:
user1_ip2_one_window = [
  ('cn=u1,ou=Identity', '/path/changePassword', '2018-02-01T00:00:00.000+0000', 6, 'IP1'), 
  ('cn=u1,ou=Identity', '/path/changePassword', '2018-02-01T00:00:01.000+0000', 6, 'IP1'), 
  ('cn=u1,ou=Identity', '/path/changePassword', '2018-02-01T00:00:03.000+0000', 6, 'IP1'),
  ('cn=u2,ou=Identity', '/path/changePassword', '2018-02-01T00:00:03.000+0000', 6, 'IP2'),
  ('cn=u2,ou=Identity', '/path/changePassword', '2018-02-01T00:00:06.000+0000', 6, 'IP2') 
]

expected_1user_ip2_one_window = [
  
  ({"start": "2018-02-01T00:00:00.000+0000", "end": "2018-02-01T00:15:00.000+0000"}, '2018-02-01T00:00:00.000+0000', '2018-02-01T00:15:00.000+0000', 5, 5, 5, 3, 0, 2)
]

user1_ip2_one_window_df = process_test_data(user1_ip2_one_window)
expected_1user_ip2_one_window_df = process_expected_data(expected_1user_ip2_one_window)

In [10]:
user1_ip2_one_window_df.write.parquet('/home/vmadmin/PycharmProjects/sw-test/caa-streamworx/data/parquet_data/server_feature_generator_tests/user1_ip2_one_window_df.parquet')
expected_1user_ip2_one_window_df.write.parquet('/home/vmadmin/PycharmProjects/sw-test/caa-streamworx/data/parquet_data/server_feature_generator_tests/expected_1user_ip2_one_window_df.parquet')

In [11]:
user1_ip1_two_window = [
  ('cn=u1,ou=Identity', '/path/changePassword', '2018-02-01T00:00:00.000+0000', 6, 'IP1'), 
  ('cn=u1,ou=Identity', '/path/changePassword', '2018-02-01T00:00:01.000+0000', 6, 'IP1'), 
  ('cn=u1,ou=Identity', '/path/changePassword', '2018-02-01T00:00:03.000+0000', 6, 'IP1'),
  ('cn=u1,ou=Identity', '/path/changePassword', '2018-02-01T00:17:00.000+0000', 6, 'IP1'), 
  ('cn=u1,ou=Identity', '/path/changePassword', '2018-02-01T00:18:01.000+0000', 6, 'IP1'), 
  ('cn=u1,ou=Identity', '/path/changePassword', '2018-02-01T00:19:03.000+0000', 6, 'IP1'),
  
]

expected_1user_ip1_two_window = [
  
  ({"start": "2018-02-01T00:00:00.000+0000", "end": "2018-02-01T00:15:00.000+0000"}, '2018-02-01T00:00:00.000+0000', '2018-02-01T00:15:00.000+0000', 3, 3, 3, 3, 0, 1), 
  ({"start": "2018-02-01T00:15:00.000+0000", "end": "2018-02-01T00:30:00.000+0000"}, '2018-02-01T00:15:00.000+0000', '2018-02-01T00:30:00.000+0000', 3, 3, 3, 3, 0, 1)
]

user1_ip1_two_window_df = process_test_data(user1_ip1_two_window)
expected_1user_ip1_two_window_df = process_expected_data(expected_1user_ip1_two_window)

In [12]:
user1_ip1_two_window_df.write.parquet('/home/vmadmin/PycharmProjects/sw-test/caa-streamworx/data/parquet_data/server_feature_generator_tests/user1_ip1_two_window_df.parquet')
expected_1user_ip1_two_window_df.write.parquet('/home/vmadmin/PycharmProjects/sw-test/caa-streamworx/data/parquet_data/server_feature_generator_tests/expected_1user_ip1_two_window_df.parquet')